In [1]:
import numpy as np
import scipy.special
import pandas as pd

#importing our plotting package
import bokeh.io
import bokeh.plotting

import os
path = os.getcwd()

#Enabling viewing Bokeh in jupyter
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv(f'{path}/data/bokeh/gfmt_sleep.csv', na_values='*')
display(df)

,participant number,gender,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence incorrect hit,confidence correct reject,confidence incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess
0,8,f,39,65,80,72.5,91.0,90.0,93.0,83.5,93.0,90.0,9,13,2
1,16,m,42,90,90,90.0,75.5,55.5,70.5,50.0,75.0,50.0,4,11,7
2,18,f,31,90,95,92.5,89.5,90.0,86.0,81.0,89.0,88.0,10,9,3
3,22,f,35,100,75,87.5,89.5,NaN,71.0,80.0,88.0,80.0,13,8,20
4,27,f,74,60,65,62.5,68.5,49.0,61.0,49.0,65.0,49.0,13,9,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,97,f,23,70,85,77.5,77.0,66.5,77.0,77.5,77.0,74.0,20,8,10
98,98,f,70,90,85,87.5,65.5,85.5,87.0,80.0,74.0,80.0,19,8,7
99,99,f,24,70,80,75.0,61.5,81.0,70.0,61.0,65.0,81.0,31,2,15
100,102,f,40,75,65,70.0,53.0,37.0,84.0,52.0,81.0,51.0,22,4,7


In [3]:
df['insomnia'] = df['sci'] <= 16 #create a column called 'insomnia', which is equal 
                                 #to the boolean value of comparing values in the 
                                 #column 'sci' to 16

df.groupby('insomnia')['percent correct'].median() #group the dataframe by the values of
                                                   #'insomnia' column, and get the median
                                                   #of the 'percent correct' value for each
                                                   #distinct element of 'insomnia'.

insomnia
False    85.0
True     75.0
Name: percent correct, dtype: float64

In [4]:
#1. Creating the 'canvas' for our plot(s).
#We must already start thinking about what we want to plot.
canvas = bokeh.plotting.figure(
         width=400,
         height=300,
         x_axis_label='confidence when correct',
         y_axis_label='confidence when incorrect'
)

In [5]:
#2. Now we need to define our data source. Let us use our DataFrame df for that.
#3. Choose the kind of glyph we need for our purposes. Let us use circles for that.
#4. Annotate which parts of the data source we would like to use and HOW.
canvas.circle(
    source=df,
    x='confidence when correct',
    y='confidence when incorrect'
)

GlyphRenderer(id='1039', ...)

In [6]:
bokeh.io.show(canvas)

In [13]:
#We can use color to add a dimension here! Let us say that we want to color the circles
#representing the insomniacs with Magenta, and non-insomniacs with Blue.
#In Bokeh, we can use this in multiple ways: 
#1. We can make multiple calls to glyphs, populating the figure in multiple steps
#(insomniacs first, non-insomniacs second or vica versa).
x_column = 'confidence when correct'
y_column = 'confidence when incorrect'

canvas = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label=x_column,
    y_axis_label=y_column
)

canvas.circle(
    source=df.loc[df['insomnia'], :], #Accessing the group of rows where the values in the
                                      #'insomnia' column are True
    x=x_column,
    y=y_column,
    color='magenta',
    legend_label='insomniacs'
)

canvas.circle(
    source=df.loc[~df['insomnia'], :], #The same as above but where the values are False
    x=x_column,
    y=y_column,
    color='blue',
    legend_label='normal sleepers'
)

GlyphRenderer(id='1467', ...)

In [14]:
bokeh.io.show(canvas)

In [9]:
#We did what we wanted to! But the legend is obstructing the view for us... We could just
#move the graph a little, but there are better ways!

canvas.legend.location = 'top_left' #Sets the position of the legend at the top left corner
                                    #of the plot.

canvas.legend.click_policy = 'hide' #Now, on click on the glyph on the legend, all the
                                    #that it represents on the plot will be hidden! Useful
                                    #when we want to look at a cluttered plot

bokeh.io.show(canvas)

In [18]:
#Now, this is a pretty cool plot, but maybe we would like to know more about certain data
#points on the plot. We cannot account possibly for every dimension, but we could make
#bokeh show us additional tooltips on hovering over the datapoint!

canvas = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label=x_column,
    y_axis_label=y_column,
    tooltips=[
        ('p-number', '@{participant number}'),
        ('gender', '@gender'),
        ('age', '@age'),
    ]
)

canvas.circle(
    source=df.loc[df['insomnia'], :],
    x=x_column,
    y=y_column,
    color='magenta',
    legend_label='insomniacs'
)

canvas.circle(
    source=df.loc[~df['insomnia'], :],
    x=x_column,
    y=y_column,
    color='blue',
    legend_label='normal sleepers'
)

canvas.legend.location = 'top_left'
canvas.legend.click_policy = 'hide'

bokeh.io.show(canvas)